# Notebook magique with AutoML

In [139]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sns as sns
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack


# Load the training data
train_data = pd.read_csv("data/train.csv")

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")

# remove outliers with quartile
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    df = df[(df[col] >= q1 - 5 * iqr) & (df[col] <= q3 + 5 * iqr)]
    return df

#train_data = remove_outliers(train_data, 'followers_count')

#reset index
#train_data = train_data.reset_index(drop=True)

Split data

In [ ]:
#split data
X_train, y_train = train_data.drop(['retweets_count'], axis=1), train_data['retweets_count']


In [140]:

#mentions is always empty -> DROP Mention
train_data = train_data.drop(['mentions'], axis=1)

#drop verified
train_data= train_data.drop(['verified'], axis=1)

#Tweet id is not relevant -> DROP Tweet id
train_data = train_data.drop(['TweetID'], axis=1)

#add a column to data which counts url
#X_train['url_count'] = X_train['urls'].str.count('http')
#X_test['url_count'] = X_test['urls'].str.count('http')
train_data= train_data.drop(['urls'], axis=1)
#add a column to data which counts hashtags
train_data= train_data.drop(['hashtags'], axis=1)
#drop the text column
train_data = train_data.drop(['text'], axis=1)


#add a column to data which makes followers/friends
train_data['followers_friends'] = train_data['followers_count'] / (train_data['friends_count'].apply(lambda x: x+1))



In [149]:
#array to df
train_data = pd.DataFrame(train_data, columns=eval_data.drop(['TweetID'], axis=1).columns)
#add columns title
train_data.head()

,0,1,2,3,4,5,6
0,0.000047,0.000000,2.553912e-04,0.055420,0.015291,0.997838,2.450137e-07
1,0.000000,0.000000,5.965140e-06,0.000124,0.001197,0.999905,7.286974e-08
2,0.000047,0.000008,1.348399e-04,0.003450,0.008408,0.998970,2.351957e-07
3,0.000000,0.000000,6.936210e-08,0.000131,0.000000,0.998641,2.414869e-07
4,0.000000,0.000000,9.680868e-04,0.003093,0.045691,0.998647,3.108682e-07


## Scaler

In [143]:
#MinMax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)


In [144]:
from flaml import AutoML
automl = AutoML()

settings = {
    "time_budget": 1800,  # total running time in seconds
    "metric": 'mae',  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
                           # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": 'regression',  # task type
    "estimator_list": ['rf'],  # list of ML learners
    "seed": 7654321,    # random seed
}

automl.fit(X_train, y_train, **settings)
# Print the best model
print(automl.model.estimator)

[flaml.automl: 11-17 08:28:22] {2600} INFO - task = regression
[flaml.automl: 11-17 08:28:22] {2602} INFO - Data split method: uniform
[flaml.automl: 11-17 08:28:22] {2605} INFO - Evaluation method: holdout
[flaml.automl: 11-17 08:28:23] {2727} INFO - Minimizing error metric: mae
[flaml.automl: 11-17 08:28:23] {2869} INFO - List of ML learners in AutoML Run: ['rf']
[flaml.automl: 11-17 08:28:23] {3164} INFO - iteration 0, current learner rf
[flaml.automl: 11-17 08:28:23] {3297} INFO - Estimated sufficient time budget=26096s. Estimated necessary time budget=26s.
[flaml.automl: 11-17 08:28:23] {3344} INFO -  at 0.4s,	estimator rf's best error=12.0668,	best estimator rf's best error=12.0668
[flaml.automl: 11-17 08:28:23] {3164} INFO - iteration 1, current learner rf
[flaml.automl: 11-17 08:28:23] {3344} INFO -  at 0.5s,	estimator rf's best error=11.8679,	best estimator rf's best error=11.8679
[flaml.automl: 11-17 08:28:23] {3164} INFO - iteration 2, current learner rf
[flaml.automl: 11-17

RandomForestRegressor(max_features=0.9568058814472833, max_leaf_nodes=11390,
                      n_estimators=7, n_jobs=-1)



AutoML.fit(self, X_train, y_train, dataframe, label, metric, task, n_jobs, log_file_name, estimator_list, time_budget, max_iter, sample, ensemble, eval_method, log_type, model_history, split_ratio, n_splits, log_training_metric, mem_thres, pred_time_limit, train_time_limit, X_val, y_val, sample_weight_val, groups_val, groups, verbose, retrain_full, split_type, learner_selector, hpo_method, starting_points, seed, n_concurrent_trials, keep_search_state, preserve_checkpoint, early_stop, append_log, auto_augment, min_sample_size, use_ray, metric_constraints, custom_hp, cv_score_agg_func, skip_transform, fit_kwargs_by_estimator, **fit_kwargs)


# Submission

In [145]:
# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")

In [146]:
#mentions is always empty -> DROP Mention

eval_data = eval_data.drop(['mentions'], axis=1)
eval_data = eval_data.drop(['hashtags'], axis=1)
eval_data = eval_data.drop(['verified'], axis=1)
eval_data = eval_data.drop(['urls'], axis=1)


#drop the text column
eval_data = eval_data.drop(['text'], axis=1)

#add a column to data which makes followers/friends
eval_data['followers_friends'] = eval_data['followers_count'] / (eval_data['friends_count'].apply(lambda x: x+1))


## Scaler

In [147]:
#MinMAX Scale all features but not the TweetID

print(eval_data.head())

eval_databis = scaler.transform(eval_data.drop(['TweetID'], axis=1))

eval_databis = pd.DataFrame(eval_databis, columns=eval_data.drop(['TweetID'], axis=1).columns)
#add tweet id
eval_data = pd.concat((eval_data['TweetID'], eval_databis), axis=1)


   favorites_count  followers_count  statuses_count  friends_count  \
0                0               85            4442            327   
1                0              427           33282            443   
2                6             1127           13111           1596   
3                2             1699           25760           2036   
4                0              249           20718            369   

       timestamp  TweetID  followers_friends  
0  1647607994000  1184643           0.259146  
1  1647601275000  1199190           0.961712  
2  1646900684000   917372           0.705698  
3  1647161294000   731754           0.834070  
4  1647724874000  1400049           0.672973  


ValueError: X has 6 features, but MinMaxScaler is expecting 7 features as input.

In [136]:
#eval data to dataframe
eval_data = pd.DataFrame(eval_data)
eval_data.head()

,TweetID,favorites_count,followers_count,statuses_count,friends_count,timestamp,followers_friends
0,1184643,0.000000,0.000006,0.000543,0.001378,0.999656,6.258045e-08
1,1199190,0.000000,0.000030,0.004067,0.001867,0.999637,2.322408e-07
2,917372,0.000049,0.000078,0.001602,0.006727,0.997615,1.704169e-07
3,731754,0.000016,0.000118,0.003148,0.008581,0.998367,2.014169e-07
4,1400049,0.000000,0.000017,0.002532,0.001555,0.999993,1.625142e-07


In [137]:
# Predict the CoverType for the evaluation dataset
#import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
#import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesRegressor

model=RandomForestRegressor(max_features=0.9568058814472833, max_leaf_nodes=11390,
                      n_estimators=7, n_jobs=-1)

model.fit(X_train, y_train)

#show top features
importances = model.feature_importances_
print(importances)



[0.74507699 0.03704467 0.03632608 0.03621806 0.12219945 0.02313475]


In [138]:

pred = model.predict(eval_data.drop(['TweetID'], axis=1))
#round to integer
pred = np.round(pred)
# Dump the results into a csv file that follows the required Kaggle template
eval_data['retweets_count'] = pred
eval_data[["TweetID", "retweets_count"]].to_csv("data/submission.csv", index=False)